#Importazione dataset.py oltre ad installazione e importazione librerie necessarie

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
percorso =  "/content/drive/MyDrive/Progetto_laboratorio"
import sys
sys.path.append(percorso)
import fiona
import os

!pip install rasterio
!pip install geopandas
!pip install matplotlib

import pandas
import numpy
import torch
import torch.nn as nn

!pip install torch torchvision
import torchvision.transforms.functional as TF
from dataset import Dataset


#Implementazione modello


In [ ]:
class DoubleConv(nn.Module):
  def __init__(self,in_channels,out_channels):
    super(DoubleConv,self).__init__()
    #eseguiamo due convoluzioni 2d consecutive, ciascuna seguita da una normalizzazione batch e funzione di attivazione ReLu
    self.conv = nn.Sequential(nn.Conv2d(in_channels,out_channels,3,1,1,bias=False),       #convoluzione 2d con kernel size 3x3, stride e padding 1
                              nn.BatchNorm2d(out_channels),                               #normalizzazione batch utile per stabilizzare e accelerare training
                              nn.ReLu(inplace=True),                                      #activation function applicata in-place
                              nn.Conv2d(out_channels,out_channels,3,1,1,bias=False),
                              nn.BatchNorm2d(out_channels),
                              nn.ReLu(inplace=True))

  def forward(self,x):
    return self.conv(x)

class UNET(nn.Module):
  def __init__(self,in_channels=6,out_channels=1, features=[64,128,256,512]):              #mettiamo 6 canali poichè per il problema di change detection dovremo passare le due immagini da analizzare concatenate sui tre canali
    super(UNET,self).__init__()                                                                                          #features sta a indicare il numero di canali di output per ogni livello della rete
    self.ups = nn.ModuleList()                                       #usiamo modulelist perchè andremo a memorizzare dei livelli convoluzionali
    self.downs = nn.ModuleList()
    self.pool = nn.MaxPool2d(kernel_size=2,stride=2)                  #pooling layer per ridurre la dimensione spaziale delle feature maps

    #parte bassa della unet, cioè si fa l'estrazione delle caratteristiche dall'immagine
    for feature in features:
      self.downs.append(DoubleConv(in_channels, feature))
      in_channels = feature

    #parte alta della unet, cioè ricostruisce l'immagine segmentata partendo dalle caratteristiche estratte dalla parte bassa. Usa strati di convoluzione trasposta

    for feature in reverse(features):
      self.ups.append(nn.ConvTranspose2d(feature*2,feature,kernel_size=2,stride=2))
      self.ups.append(DoubleConv(feature*2,feature))

    self.bottleneck = DoubleConv(features[-1],features[-1]*2)                           #prende l'ultimo set di features maps e lo trasforma in un set con li doppio dei canali per catturare caratteristiche piu complesse
    self.final_conv = NN.Conv2d(features[0],out_channels,kernel_size=1)                 #strato di convoluzione con kernel 1x1 per ridurre i canali al numero di classi di output desiderato


#definisce come i dati di input x vengono trasformati attraverso la cnn per produrre l'output
  def forward(self,x):
    skip_connections = []                             #lista per memorizzare le skip connections,quest'ultime aiutano la stabilità e la velocità di convergenza del modello e permettono di utilizzare informazioni a diversi livelli di astrazione per identificare grandi e piccoli cambiamenti

    #itera sui blocchi di convoluzione della parte bassa
    for down in self.downs:
      x = down(x)                                 #applica down (cioè double convolution) su x
      skip_connections.append(x)
      x = self.pool(x)                           #per ridurre la dimensione della feature maps

    x = self.bottleneck(x)                             #applichiamo il bottleneck e poi invertiamo l'ordine delle skip connections per utilizzarle nella parte alta
    skip_connections = skip_connections[::-1]

    #itera sulla parte alta , quindi sui blocchi di convoluzione trasposta però in coppie
    for idx in range(0,len(self.ups),2):                        #iteriamo in passi di 2 perchè ogni livello ha la componente di conv transpose e doubleconv e a noi interessa solo la prima
      x = self.ups[idx](x)                                      #applica la conv transpose a x(upsampling)
      skip_connections = skip_connections[idx//2]
      if x.shape != skip_connections.shape:                     #verifichiamo se dimensioni corrispondono e in caso contrario ridimensioniamo x
        x = TF.resize(x,size=skip_connections.shape[2:])

      concat_skip = torch.cat((skip_connection,x),dim=1)       #concateniamo x con la skip connection lungo la dimensione del canale per combinare informazioni dettagliate della parte bassa con quelle della parte upsampled
      x = self.ups[idx+1](concat_skip)                         #applichiamo il blocco doubleconv

    return self.final_conv(x)                                #applichiamo uno strato di convoluzione con kernel 1x1 per ridurre il numero di canali all'output che ci serve



